## Few-shot prompt templates

In [3]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate


examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [5]:
# Create a formatter for the few-shot examples

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

example_prompt.format(**examples[0])

'Question: Who lived longer, Muhammad Ali or Alan Turing?\n\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n'

In [6]:
# Feed examples and formatter to FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

prompt.format(input="Who was the father of Mary Ball Washington?")

'Question: Who lived longer, Muhammad Ali or Alan Turing?\n\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\n\nQuestion: When was the founder of craigslist born?\n\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\n\nQuestion: Who was the maternal grandfather of George Washington?\n\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was

#### Using an example selector

In [11]:
#  Feed examples into ExampleSelector
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)


prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

prompt.format(input="Who was the father of Mary Ball Washington?")

'Question: Who was the maternal grandfather of George Washington?\n\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the father of Mary Ball Washington?\nIntermediate answer: The father of Mary Ball Washington was Joseph Ball.\nSo the final answer is: Joseph Ball\n\n\nQuestion: Who was the father of Mary Ball Washington?'

In [14]:
openai_llm = ChatOpenAI()
chain = prompt | openai_llm

chain.invoke(input="Who was the father of Hanuman?")

AIMessage(content="Are follow up questions needed here: Yes.\nFollow up: In Hindu mythology, who was the mother of Hanuman?\nIntermediate answer: The mother of Hanuman was Anjana.\nFollow up: Was there a significant event related to Hanuman's birth?\nIntermediate answer: Yes, Hanuman was born to Anjana through the blessings of the god Vayu, the god of wind.\nSo the final answer is: Vayu", response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 95, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16471ecd-7c33-4085-8423-ac455aa7b253-0')

In [15]:
chain.invoke(input="Who was the prime ministe of india?")

AIMessage(content='Are follow up questions needed here: Yes\nFollow up: Can you provide the names of some former Prime Ministers of India?\nIntermediate answer: Some former Prime Ministers of India include Jawaharlal Nehru, Indira Gandhi, Rajiv Gandhi, Atal Bihari Vajpayee, and Manmohan Singh.\nFinal answer: The Prime Minister of India as of September 2021 is Narendra Modi.', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 100, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9f3450db-b12f-4a6d-9c05-b1472dda2402-0')